# Proyecto Oily Giant 🏭🔥

## 1. Preparación de los Datos

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
import matplotlib.pyplot as plt

try:
    df0 = pd.read_csv('geo_data_0.csv')
    df1 = pd.read_csv('geo_data_1.csv')
    df2 = pd.read_csv('geo_data_2.csv')
    print("Datos cargados correctamente, con metodo 1.")
    
except FileNotFoundError:
    df0 = pd.read_csv('/datasets/geo_data_0.csv')
    df1 = pd.read_csv('/datasets/geo_data_1.csv')
    df2 = pd.read_csv('/datasets/geo_data_2.csv')
    print("Datos cargados con método 2 (ruta alternativa)")
    
# Verificar estructura de datos
print(df0.info(), df0.describe())
print(df1.info(), df1.describe())
print(df2.info(), df2.describe())

Datos cargados correctamente, con metodo 1.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None                   f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.248248      44.288691
min        -1.408605      -0.848218     -12.088328       0.000000
25%        -0.072580      -0.200881       0.287748      56.497507
50%         0.502360       0.250252       2.515969      91.849972
75%         1.073581       0.700646       4.715088     

Conclusiones iniciales:

* Región 2 tiene el promedio de reservas más alto (95.0) y la mediana más alta, lo cual es muy prometedor.

* Región 0 tiene una media apenas inferior, pero también es una buena candidata.

* Región 1 tiene significativamente menor promedio y mediana, lo que la hace menos atractiva desde el punto de vista del volumen esperado de reservas.

In [9]:
from sklearn.model_selection import train_test_split

# Función para dividir datos
def dividir_datos(df):
    X = df[['f0', 'f1', 'f2']]
    y = df['product']
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=42)
    return X_train, X_valid, y_train, y_valid

X_train0, X_valid0, y_train0, y_valid0 = dividir_datos(df0)
X_train1, X_valid1, y_train1, y_valid1 = dividir_datos(df1)
X_train2, X_valid2, y_train2, y_valid2 = dividir_datos(df2)

## 🧪 2. Entrenamiento y evaluación del modelo

Entrenamos un modelo de regresión lineal y evaluamos su rendimiento.

In [10]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Función para entrenar el modelo
def entrenar_modelo(X_train, y_train, X_valid, y_valid):
    modelo = LinearRegression()
    modelo.fit(X_train, y_train)
    predicciones = modelo.predict(X_valid)
    mse = mean_squared_error(y_valid, predicciones)
    rmse = mse ** 0.5
    return modelo, predicciones, rmse

modelo0, predicciones0, rmse0 = entrenar_modelo(X_train0, y_train0, X_valid0, y_valid0)
modelo1, predicciones1, rmse1 = entrenar_modelo(X_train1, y_train1, X_valid1, y_valid1)
modelo2, predicciones2, rmse2 = entrenar_modelo(X_train2, y_train2, X_valid2, y_valid2)

print(f"RMSE Región 0: {rmse0}")
print(f"RMSE Región 1: {rmse1}")
print(f"RMSE Región 2: {rmse2}")

RMSE Región 0: 37.75660035026169
RMSE Región 1: 0.8902801001028854
RMSE Región 2: 40.145872311342174


* Región 1 tiene un RMSE extraordinariamente bajo (0.89), lo que sugiere que el modelo predice casi perfectamente el volumen de reservas.

* Regiones 0 y 2 tienen RMSEs muy altos (≈38–40), lo que indica baja precisión predictiva en esas regiones.

In [11]:
## 3. Cálculo de ganancias

Usando los 200 pozos

SyntaxError: invalid syntax (3668427494.py, line 3)

In [ ]:
def calcular_ganancia(predicciones, y_valid):
    pozos_seleccionados = predicciones.argsort()[-200:]  # Selección de los 200 mejores
    ganancia_total = y_valid.iloc[pozos_seleccionados].sum() * 4500
    return ganancia_total

ganancia0 = calcular_ganancia(predicciones0, y_valid0)
ganancia1 = calcular_ganancia(predicciones1, y_valid1)
ganancia2 = calcular_ganancia(predicciones2, y_valid2)

print(f"Ganancia Región 0: ${ganancia0}")
print(f"Ganancia Región 1: ${ganancia1}")
print(f"Ganancia Región 2: ${ganancia2}")

* Región 0 proporciona la mayor ganancia estimada, superando por más de 8 millones de dólares a las otras dos regiones.

* A pesar de que el modelo en Región 1 tiene el RMSE más bajo (es decir, es más preciso), el volumen promedio de reservas en esa región es menor, lo que reduce el beneficio total.

* Región 2 tiene el mayor volumen promedio, pero el modelo predice con menos precisión, lo que lleva a una ganancia inferior comparada con Región 0.

## 4. Análisis de riesgos con bootstrapping

Simulación de 1000 muestras para evaluar el riesgo:

In [ ]:
def bootstrap(predicciones, y_valid, n=1000):
    np.random.seed(42)
    ganancias = []
    for _ in range(n):
        muestra = np.random.choice(predicciones, 200, replace=True)
        ganancia_muestra = y_valid.iloc[muestra.argsort()[-200:]].sum() * 4500
        ganancias.append(ganancia_muestra)
    
    ganancia_media = np.mean(ganancias)
    intervalo_confianza = (np.percentile(ganancias, 2.5), np.percentile(ganancias, 97.5))
    riesgo_perdida = (np.sum(np.array(ganancias) < 100_000_000) / n) * 100
    return ganancia_media, intervalo_confianza, riesgo_perdida

ganancia_media0, conf_0, riesgo0 = bootstrap(predicciones0, y_valid0)
ganancia_media1, conf_1, riesgo1 = bootstrap(predicciones1, y_valid1)
ganancia_media2, conf_2, riesgo2 = bootstrap(predicciones2, y_valid2)

print(f"Región 0 - Ganancia media: ${ganancia_media0}, Intervalo de confianza: {conf_0}, Riesgo: {riesgo0}%")
print(f"Región 1 - Ganancia media: ${ganancia_media1}, Intervalo de confianza: {conf_1}, Riesgo: {riesgo1}%")
print(f"Región 2 - Ganancia media: ${ganancia_media2}, Intervalo de confianza: {conf_2}, Riesgo: {riesgo2}%")

## Conclusion Final

La mejor región para la extracción será aquella con: ✅ Mayor ganancia media ✅ Menor riesgo de pérdidas (<2.5%)

Aunque la Región 2 parece tener la mayor ganancia media, el hecho de que el riesgo sea del 100% en todas las regiones implica que ninguna alcanza el punto de equilibrio ($100 millones) en ninguna simulación.

Pero estos resultados no son confiables si el bootstrapping no fue aplicado correctamente. Debes revisar y corregir el muestreo y la lógica de selección de pozos dentro del ciclo.